In [5]:
import os
import pandas as pd

train_data_path ="~/aiffel/240624-dlthon/data/train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [6]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback
from sklearn.preprocessing import LabelEncoder

In [21]:
train_data_path ="~/aiffel/240624-dlthon/data/train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [22]:
train_data

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
3945,3945,기타 괴롭힘 대화,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...
3946,3946,갈취 대화,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...
3947,3947,직장 내 괴롭힘 대화,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...
3948,3948,갈취 대화,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...


In [14]:
# 중복된 행을 찾기
duplicates = train_data.duplicated()
print(duplicates)

# 중복된 행의 개수를 세기
num_duplicates = train_data.duplicated().sum()
print(f'Number of duplicate rows: {num_duplicates}')

0       False
1       False
2       False
3       False
4       False
        ...  
3945    False
3946    False
3947    False
3948    False
3949    False
Length: 3950, dtype: bool
Number of duplicate rows: 0


In [36]:
print(train_data['conversation'].nunique())

3846


In [37]:
# 중복된 행을 찾기
val_counts = train_data['conversation'].value_counts()

print(val_counts)

저기요. 잠깐만요\n매일 왜이러세요. 전 할말 없어요\n그 쪽이 좋아서요\n전 관심 없어요. 오지 마세요\n한 번만 만나주세요.\n제발 이러지 마세요. 전 남자친구 있어요\n결혼한 건 아니잖아요. 딱 한번만 데이트해요\n내가 왜 그쪽이랑 만나요?\n그 쪽이 내 스타일이에요\n난 아니거든요.                                                                                                                                                        2
야 석탄?\n 내가 왜 석탄이야?\n 얼굴이 까마니까 석탄이지.\n 그만해.\n 너 라이터 불붙이면 아주 잘 타겠다.\n 가 비켜.\n 아주 콧구멍 커지니까 이제 연탄이네.\n 비키라고.\n 야 연탄?\n 제발 좀 그만해. 나 정말 힘들어.                                                                                                                                                                                    2
진용아 내 내 가방 돌려줘\n아하하 진용아? 가방 돌려줄까?\n노 놀리지마\n노 놀리지마\n하아.\n한숨 쉰 거야?\n아 아 아아니야.\n아 아 아 아니야? 하하하하\n아아.\n왜? 기분 나빠?                                                                                                                                                                                                              2
김대리. 이번 주간 회의 발표 준비좀 해줘.\n저 말입니까.?\n응. 너가 해줘\n과장님 저번

In [38]:
print(train_data[train_data['conversation'] == "저기요. 잠깐만요\n매일 왜이러세요. 전 할말 없어요\n그 쪽이 좋아서요\n전 관심 없어요. 오지 마세요\n한 번만 만나주세요.\n제발 이러지 마세요. 전 남자친구 있어요\n결혼한 건 아니잖아요. 딱 한번만 데이트해요\n내가 왜 그쪽이랑 만나요?\n그 쪽이 내 스타일이에요\n난 아니거든요."])

     idx      class                                       conversation
262  262  기타 괴롭힘 대화  저기요. 잠깐만요\n매일 왜이러세요. 전 할말 없어요\n그 쪽이 좋아서요\n전 관심...
932  932  기타 괴롭힘 대화  저기요. 잠깐만요\n매일 왜이러세요. 전 할말 없어요\n그 쪽이 좋아서요\n전 관심...


In [39]:
train_data.drop_duplicates(subset = ['conversation'], inplace=True)
print(len(train_data))

3846


In [ ]:
train

In [33]:
stopwords = {'이', '그', '저', '의', '가', '을', '를', '은', '는', '에', '와', '과', '도', '으로', '로', '에서', '에게', '뿐'}

In [35]:
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

In [40]:
import re

def remove_stopwords(sentence, stopwords):
    # 불용어 리스트를 정규표현식에 맞게 포맷합니다.
    stopwords_regex = r'\b(' + '|'.join(stopwords) + r')\b'
        
    # 정규표현식을 사용하여 불용어를 제거합니다.
    sentence_without_stopwords = re.sub(stopwords_regex, '', sentence)
    
    return sentence_without_stopwords

# 예제 문장과 불용어 리스트
sentence = "이 문장에서 '이'와 '가'를 제거해 봅시다."
stopwords = ['이', '가', '을', '를']

# 불용어 제거 함수 호출
filtered_sentence = remove_stopwords(sentence, stopwords)
print(filtered_sentence)


 문장에서 ''와 '' 제거해 봅시다.
